In [ ]:
!unzip data.zip

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import pickle
import os

import sys
# sys.path.append(os.path.join('..', '..'))

In [ ]:
# Seed for all PRNGs used in this assignment.
SEED = 1

# ---------- TRAIN-TEST SPLIT ----------
# Minimum number of images per class to consider for train and test.
min_img_per_class = 3
# If class contains min_img_per_class, fraction of total images in class to keep for testing.
test_frac = 0.2
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
target_size = (220, 220)
batch_size = 200

In [ ]:
train_path = os.path.join('data', 'train')

train_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    # rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1
).flow_from_directory(
    train_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse'
)

Found 5783 images belonging to 901 classes.


In [ ]:
#Checkpoint
checkpoint_path = "train_ckpt/cp.ckpt"
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1)

In [ ]:
# Callback for early stopping
# es_callback = keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     min_delta=0.0001,
#     patience=5,
#     restore_best_weights=True
# )

In [ ]:
# Hyperparameters
epochs = 20
learning_rate = 0.03
validation_split = 0.1
# Math works out so that one epoch equals one pass through the training data.
train_steps_per_epoch = train_batches.n // batch_size

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (7, 7), activation='relu', strides=(2, 2), padding='same', input_shape=(target_size[0], target_size[1], 3), name='conv1'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool1'),
    keras.layers.BatchNormalization(name='norm1'),
    
    keras.layers.Conv2D(64, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv2a'),
    keras.layers.Conv2D(192, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv2'),
    keras.layers.BatchNormalization(name='norm2'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool2'),
    
    keras.layers.Conv2D(192, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv3a'),
    keras.layers.Conv2D(384, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv3'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool3'),
    
    keras.layers.Conv2D(384, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv4a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv4'),
    
    keras.layers.Conv2D(256, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv5a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv5'),
    
    keras.layers.Conv2D(256, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv6a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv6'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', name='pool4'),

    keras.layers.Flatten(name='flatten'),
    
    keras.layers.Dense(32 * 128, activation='relu', name='fc1'),
    keras.layers.Dropout(0.2, name='drop1'),
    keras.layers.Dense(32 * 128, activation='relu', name='fc2'),
    keras.layers.Dropout(0.2, name='drop2'),
    keras.layers.Dense(128, activation='relu', name='fc3'),
    
    keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name='l2')
], name='zeiler_fergus')

In [ ]:
model.summary()

Model: "zeiler_fergus"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 110, 110, 64)      9472      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 55, 55, 64)        0         
_________________________________________________________________
norm1 (BatchNormalization)   (None, 55, 55, 64)        256       
_________________________________________________________________
conv2a (Conv2D)              (None, 55, 55, 64)        4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 55, 55, 192)       110784    
_________________________________________________________________
norm2 (BatchNormalization)   (None, 55, 55, 192)       768       
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 28, 28, 192)     

In [ ]:
model.compile(
    optimizer=keras.optimizers.SGD(lr=learning_rate),
    loss=tfa.losses.TripletSemiHardLoss(margin=0.2)
)

In [ ]:
# %%script echo "Comment line with %%script echo to run this cell."

history = model.fit(
    train_batches,
    epochs=epochs,
    steps_per_epoch=train_steps_per_epoch,
    callbacks=[cp_callback]
)

In [ ]:
#get the latest checkpoint file
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)

#Create a new model instance
model_latest_checkpoint = create_model()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)


In [ ]:
test_path = os.path.join('data', 'test')

train_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
    # rotation_range=20,
    # width_shift_range=0.1,
    # height_shift_range=0.1
).flow_from_directory(
    test_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse'
)

In [ ]:
# Evaluate the network
results = model.predict(test_batches)

In [ ]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

x=np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
y=np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
print(x.shape)
print(y.shape)
y = y.astype(int)

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, label in zip(test_batches):
    [out_m.write(str(label) + "\n")]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass


In [ ]:
%%script echo "Comment line with %%script echo to run this cell."

model.save('model.h5')

In [ ]:
%%script echo "Comment line with %%script echo to run this cell."

with open('history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
model = keras.models.load_model('model.h5')

In [ ]:
with open('history.pickle', 'rb') as f:
    history = pickle.load(f)